# Библиотеки и CUDA

In [ ]:
import torch

In [5]:
from transformers import AutoProcessor, AutoModelForTextToWaveform

In [4]:
print(torch.cuda.is_available())

False


# Модуль Б. Разработка модели машинного обучения 

In [6]:
processor = AutoProcessor.from_pretrained('suno/bark-small') # хз что за параша
model_sb = AutoModelForTextToWaveform.from_pretrained('suno/bark-small') # моделька

c:\projects\tts1\.venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aaron\.cache\huggingface\hub\models--suno--bark-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
